In [138]:
import os
import numpy as np
import torch
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader
from torchvision import utils
from skimage import io
import matplotlib.pyplot as plt

data_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.RandomHorizontalFlip(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        transforms.RandomSizedCrop(224),
])

data_dir = '/Users/zhaohe/workspace/data/lung_training_data'

In [139]:
class LungDataset(Dataset):
    def __init__(self, root, transform):
        self.root = root
        self.transform = transform
        self.IMAGE_EXT = ['.png', 'jpeg', 'jpg', 'bmp']
        train_list = {}
        
        for x in ['train', 'val']:
            train_list[x] = []
            for dirpath, dnames, fnames in os.walk(os.path.join(self.root, x)):
                for f in fnames:
                    if f.endswith(tuple(self.IMAGE_EXT)):
                        train_list[x].append(os.path.join(dirpath, f))

        self.images = train_list['train']
        self.targets = train_list['val']
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image = io.imread(self.images[idx])
        target = io.imread(self.targets[idx])
       
        w,h = image.shape
        img = np.empty((3, w, h), dtype=np.float32)
        targ = np.empty((3, w, h), dtype=np.float32)

        for i in range(3):
            img[i,:,:] = image
            targ[i,:,:] = target 
        
        if self.transform:
            img = self.transform(img)
            targ = self.transform(targ)
            
        return [img, targ]

    
    def view(self, idx):
        fig = plt.figure()
        img, target = self.__getitem__(idx)
        print(type(img))
        
        ## For plt Dim must be (H,W,C)
        plt.imshow(np.transpose(img, (1,2,0)))
#         plt.imshow(img)
        plt.show()

        plt.imshow(np.transpose(target, (1,2,0)))
#         plt.imshow(target)
        plt.show()

In [140]:
lung424 = LungDataset(root=data_dir, transform=None)

lung424.view(363)

TypeError: 'builtin_function_or_method' object is not subscriptable

In [117]:
## TODO: wrap data with DataLoader
LungDataLoader = DataLoader(lung424, shuffle=True, batch_size=5)

for idx, data in enumerate(LungDataLoader):
    print(idx)
    img, target = data
    print(img.size())

0
torch.Size([5, 3, 192, 192])
1
torch.Size([5, 3, 192, 192])
2
torch.Size([5, 3, 192, 192])
3
torch.Size([5, 3, 192, 192])
4
torch.Size([5, 3, 192, 192])
5
torch.Size([5, 3, 192, 192])
6
torch.Size([5, 3, 192, 192])
7
torch.Size([5, 3, 192, 192])
8
torch.Size([5, 3, 192, 192])
9
torch.Size([5, 3, 192, 192])
10
torch.Size([5, 3, 192, 192])
11
torch.Size([5, 3, 192, 192])
12
torch.Size([5, 3, 192, 192])
13
torch.Size([5, 3, 192, 192])
14
torch.Size([5, 3, 192, 192])
15
torch.Size([5, 3, 192, 192])
16
torch.Size([5, 3, 192, 192])
17
torch.Size([5, 3, 192, 192])
18
torch.Size([5, 3, 192, 192])
19
torch.Size([5, 3, 192, 192])
20
torch.Size([5, 3, 192, 192])
21
torch.Size([5, 3, 192, 192])
22
torch.Size([5, 3, 192, 192])
23
torch.Size([5, 3, 192, 192])
24
torch.Size([5, 3, 192, 192])
25
torch.Size([5, 3, 192, 192])
26
torch.Size([5, 3, 192, 192])
27
torch.Size([5, 3, 192, 192])
28
torch.Size([5, 3, 192, 192])
29
torch.Size([5, 3, 192, 192])
30
torch.Size([5, 3, 192, 192])
31
torch.Size([5, 